In [4]:
#import required packages
import requests
import json

import numpy as np
import pandas as pd

import pandas_profiling
from pandas_profiling import ProfileReport
from flatten_json import flatten
#from pandas_profiling.utils.cache import cache_file

# Building flat table from multiple tables

## Table 1: Creator Info

In [2]:
query  = """
{
    creators(first: 1000, orderBy: netSalesInETH, orderDirection: desc)
    {
    id
    netSalesInETH
    netSalesPendingInETH
    netRevenueInETH
    netRevenuePendingInETH
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})
#print(r.status_code)
#print(r.text)

# convert json into list
json_data = json.loads(r.text)
extracted_data = json_data['data'][list(json_data['data'].keys())[0]]

#create a empty dataframe to store the extracted data
# ref to https://www.kite.com/python/answers/how-to-create-an-empty-dataframe-with-column-names-in-python
df_column_names = list(extracted_data[0].keys())
wrangled_df = pd.DataFrame(columns = df_column_names)

# store each entry from the extracted data into the empty data frame
for i in range(len(extracted_data)):
    wrangled_df = wrangled_df.append(extracted_data[i], ignore_index = True)
    
creator_df = wrangled_df
creator_df.head()

,id,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH
0,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.4,0,2224,0
1,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.178945,0.1,1623.6517,1
2,0x484ec62385e780f2460feac34864a77ba5a18134,513.4,0,604,0
3,0x0624d062ae9dd596de0384d37522cde46cd500d6,425,0,500,0
4,0x0f441cfad93287109f5ef834bf52f4aaaa8d8ffa,380.005263175,0,447.0650155,0


In [3]:
creator_df.describe().T

,count,unique,top,freq
id,1000,1000,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1
netRevenueInETH,1000,913,6.8,14
netRevenuePendingInETH,1000,33,0,963
netSalesInETH,1000,906,8,15
netSalesPendingInETH,1000,32,0,963


## Table 2: NFT info part I 

In [4]:
query  = """
{
    creators(first: 1000, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {
        id
        dateMinted
        netSalesInETH
        netSalesPendingInETH
        netRevenueInETH
        netRevenuePendingInETH
        isFirstSale
        percentSplit
        lastSalePriceInETH
        }

    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)
extracted_data = json_data['data'][list(json_data['data'].keys())[0]]



# store each entry from the extracted data into the empty list
wrangled_list = list()
for i in range(len(extracted_data)):
    wrangled_list.append(extracted_data[i])

    
    
# repeat above steps going 1 level deeper into the nest    
df2_column_names = list(wrangled_list[0]['nfts'][0].keys())
wrangled_df2 = pd.DataFrame(columns = df2_column_names)

for i in range(len(wrangled_list)):
    wrangled_df2 = wrangled_df2.append(wrangled_list[i]["nfts"])

nft_df1 = wrangled_df2
nft_df1.head()

,dateMinted,id,isFirstSale,lastSalePriceInETH,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH,percentSplit
0,1618452786,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,False,2224,1890.4,12677.174465,2224,14914.3229,None
0,1634633808,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,False,49.5002,42.07517,158.7885,49.5002,186.81,None
1,1634808905,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995,False,33,3.3,0,33,0,None
2,1634809019,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100997,True,None,0,0,0,0,None
3,1634809215,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100998,True,None,0,0,0,0,None


In [5]:
nft_df1.describe().T

,count,unique,top,freq
dateMinted,18458,18007,1628989515,8
id,18458,18458,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,1
isFirstSale,18458,2,False,12657
lastSalePriceInETH,12657,1566,1,1118
netRevenueInETH,18458,1916,0,5803
netRevenuePendingInETH,18458,3160,0,12750
netSalesInETH,18458,1714,0,5803
netSalesPendingInETH,18458,3113,0,12750
percentSplit,821,1,{},821


## Table 3 NFT History

In [6]:
query  = """
{
    creators(first: 1000, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {     
        nftHistory
            {
            id
            event
            date
            amountInETH
            amountInTokens
            marketplace
            }
        }

    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)
extracted_data = json_data['data'][list(json_data['data'].keys())[0]]

# store each entry from the extracted data into the empty list
wrangled_list = list()
for i in range(len(extracted_data)):
    wrangled_list.append(extracted_data[i])

    
    
# repeat above steps going 1 level deeper into the nest    
wrangled_list2 = list()
#df2_column_names = list(wrangled_list[0]['nfts'][0].keys())
#wrangled_df2 = pd.DataFrame(columns = df2_column_names)

for i in range(len(wrangled_list)):
    wrangled_list2.append(wrangled_list[i]["nfts"])


df_column_names = list(wrangled_list2[0][0]['nftHistory'][0].keys())
wrangled_df = pd.DataFrame(columns = df_column_names)

#wrangled_df
for i in range(len(wrangled_list2[0])):
    wrangled_df = wrangled_df.append(wrangled_list2[0][i]["nftHistory"])

nfthist_df = wrangled_df
nfthist_df.head()


,amountInETH,amountInTokens,date,event,id,marketplace
0,444,None,1618595528,Bid,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...,Foundation
1,2,None,1618512578,Listed,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...,Foundation
2,122,None,1618527555,Bid,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...,Foundation
3,611,None,1618598693,Bid,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...,Foundation
4,1337,None,1618600772,Bid,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...,Foundation


In [7]:
nfthist_df.describe().T

,count,unique,top,freq
amountInETH,33,31,2224,2
amountInTokens,0,0,NaN,NaN
date,36,36,1618595528,1
event,36,6,Bid,31
id,36,36,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...,1
marketplace,34,1,Foundation,34


## Building Connector Between Table 1 and Table 2

In [5]:
query  = """
{
    creators(first: , orderBy: netSalesInETH, orderDirection: desc)
    {
    id
    nfts
        {
        id
        }
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})
#print(r.status_code)
#print(r.text)

# convert json into list
json_data = json.loads(r.text)

creator_dict = json_data['data']['creators']
df_column_names = ['creator_id','nft_id']
creator_nft_df = pd.DataFrame(columns = df_column_names)

for creator_number in range(len(creator_dict)):
    
    creator_list = list()
    creator_str = json_data['data']['creators'][creator_number]['id']
    creator_list += [creator_str]
    creator_df = pd.DataFrame(creator_list, columns = ['creator_id'])
    
    
    nft_data = json_data['data']['creators'][creator_number]['nfts']
    nft_list = list()
    for nft_number in range(len(nft_data)):
        nft_str = json_data['data']['creators'][creator_number]['nfts'][nft_number]['id']
        nft_list += [nft_str]
        
    nft_df = pd.DataFrame(nft_list,columns = ['nft_id'])
    
    
    combined_df = pd.concat([creator_df, nft_df], axis = 1)
    combined_df.fillna(method='ffill', inplace = True)
    
    creator_nft_df = pd.concat([creator_nft_df, combined_df])
    
creator_nft_df.describe().T

,count,unique,top,freq
creator_id,18465,1000,0xf06bed3f0dad7932d8d00fe48c36751f5c10be23,100
nft_id,18459,18459,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,1


In [78]:
creator_nft_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18465 entries, 0 to 17
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   creator_id  18465 non-null  object
 1   nft_id      18459 non-null  object
dtypes: object(2)
memory usage: 432.8+ KB


In [80]:
creator_nft_df.shape

(18465, 2)

In [7]:
#datapath = '/data'
#creator_nft_df.to_csv('creator_nft_df.csv', index = False)

## Building Connector Between Table 2 and Table 3

In [68]:
query  = """
{
    creators(first: 5, orderBy: netSalesInETH, orderDirection: desc)
    {
    nfts
        {
        id
        nftHistory
            {
            id
            }
        }
    }
}
"""

url = 'https://api.thegraph.com/subgraphs/name/f8n/fnd'
r = requests.post(url, json={'query': query})

# convert json into list
json_data = json.loads(r.text)

flat_dict = flatten(json_data['data'])
flat_df = pd.DataFrame(flat_dict, index = [0])
flat_df

## update the id_vars and value name here
flat_df2 = pd.melt(flat_df,id_vars = "creators_0_nfts_0_id", value_name = "nftHistoryid")
flat_df2.pop("variable")
table23_connector = flat_df2
table23_connector.head()

,creators_0_nfts_0_id,nftHistoryid
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...


In [40]:
json_data['data']

{'creators': [{'id': '0xf74d1224931afa9cf12d06092c1eb1818d1e255c',
   'nfts': [{'id': '0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437',
     'nftHistory': [{'id': '0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd0bb891e6d372eb4d70dd-171-Bid'},
      {'id': '0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e59004711ca4ee17d45e5d-305-Listed'},
      {'id': '0x0708e2f65b149a1b6b429dbe366bd590786e86724c810badadec435715075140-95-Bid'},
      {'id': '0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057fafb7047c905ff67d1dd-17-Bid'},
      {'id': '0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5e829d07cd8e70fbf1f11-290-Bid'},
      {'id': '0x0dfd065060f31f68adc38a0a473c94e4e0783fa9cfe5937bf917e1a3d8132f9a-379-Bid'},
      {'id': '0x0f44b22633d58699c089507fe63e84a20084ce39771cb8329e7604f41ea3de6a-60-Bid'},
      {'id': '0x0f908b44547418dea157dec80187f372d0a99527d22369689c2fc761ddb17f85-287-Bid'},
      {'id': '0x1b0e551f104b1c9cd5359e84626afe7b5ca239a66134476485e1b428bfce9561-345-Bid'},
      {'id': '0x2302

In [20]:
json_data['data']['creators'][1]['nfts'][1]['nftHistory']

[{'id': '0x3142563956f478b4eb3de2fa6fce04045209a47599be7a9f5afedbd0eba218a8-34-Listed'},
 {'id': '0x32dcc2308038ee635364cb74e0feef36a27ab2aae3a81b21a2278f2a53e4e560-288-PriceChanged'},
 {'id': '0x3399b1a38afb7807078784cd1c48e08d674db23c55c40985fb01042ed8a22daf-143-PriceChanged'},
 {'id': '0x5b998a3c5703370f3fc7067afff272c95740bc3b4a0f2b574c112dcbcec07067-199-Transferred'},
 {'id': '0x792fca1d59658cc25ae64096ec37e1f2df9cc8ffeab6a770af2ae54c2d5b6074-134-Minted'},
 {'id': '0xb8f7c6838063abe6467b6a75694e04b7ba317c9ebb166267ce0025e762efc139-244-Bid'},
 {'id': '0xf2de3d6d6a0317eda6ecc8a59c67bc0ad7b698ca248c31b59ef838124bc9f744-2-Settled'},
 {'id': '0xf2de3d6d6a0317eda6ecc8a59c67bc0ad7b698ca248c31b59ef838124bc9f744-2-Sold'}]

In [64]:
creator_dict = json_data['data']['creators']
df_column_names = ['nft_id','nft_history_id']
nft_full_df = pd.DataFrame(columns = df_column_names)

#for creator_number in range(len(creator_dict)):
    
#    creator_list = list()
#    creator_str = json_data['data']['creators'][creator_number]['id']
#    creator_list += [creator_str]
#    creator_df = pd.DataFrame(creator_list, columns = ['creator_id'])
creator_number = 1    
    
nft_data = json_data['data']['creators'][creator_number]['nfts']
nft_list = list()
for nft_number in range(len(nft_data)):
    nft_str = json_data['data']['creators'][creator_number]['nfts'][nft_number]['id']
    nft_list += [nft_str]
    nft_df = pd.DataFrame(nft_list,columns = ['nft_id'])
                
    nft_history_data = json_data['data']['creators'][creator_number]['nfts'][nft_number]['nftHistory']
    nft_history_list = list()
    for nft_history_number in range(len(nft_history_data)):
        nft_history_str = json_data['data']['creators'][creator_number]['nfts'][nft_number]['nftHistory'][nft_history_number]['id']
        nft_history_list += [nft_history_str]
#        print(nft_history_list)

    nft_history_df = pd.DataFrame(nft_history_list, columns = ['nft_history_id'])
    combined_df = pd.concat([nft_df, nft_history_df], axis = 1)
#        combined_df.fillna(method='ffill', inplace = True)
    
    nft_full_df = pd.concat([nft_full_df,combined_df])

nft_full_df.shape
nft_full_df.to_csv('nft_full_df.csv', index = False)

In [62]:
json_data['data']['creators'][1]['nfts'][0]['nftHistory']

[{'id': '0x1587f580218abb040ebdc502d7eb0b78265f34e7b8acbe87da6123084a4e7e06-103-Settled'},
 {'id': '0x1587f580218abb040ebdc502d7eb0b78265f34e7b8acbe87da6123084a4e7e06-103-Sold'},
 {'id': '0x1febc90893e04d36e990d84d305e27b9298511b4b01b51f9aa4a9aac13b50a16-566-Minted'},
 {'id': '0x284aa8be47571cf49ec7155e4c9c1b0a052f1199e87f5d0d050642993cac123d-145-Bid'},
 {'id': '0x325741552edb163155f74afd0f6d2c1be344005da4641bfc95110a4fc870d26f-407-Listed'},
 {'id': '0x36aae5236169f388b56962ba426c4163aac25592083e29d548ccc98bef6a1ab7-140-Bid'},
 {'id': '0x947d74553385765d1789d02f290ce01916622428274b592a4aa06f5f8fcc4f08-500-Bid'},
 {'id': '0xa0a30483c68eb3b0381d8d60a183cdae24c1f2ba96b9134d7bc2a39c78000d0f-445-Bid'},
 {'id': '0xb6d38adb920650910d1d7221a33735c943f649c9ee14c90be52db813e40b86fe-206-Bid'},
 {'id': '0xc3b4cb4484652045703c6e3713775e1026c5d0279ea9ff16cfafbd962a27d386-513-Bid'},
 {'id': '0xe04ccf548a28c730ea5495c2879a8dc395cc614cee208b7e2b16f7304ca6e24a-543-Bid'},
 {'id': '0xfbfb7aa4b39e426f4ae50

## method from bottom up

In [81]:
nft_data = json_data['data']['creators'][1]['nfts'][0]
nft_list = [json_data['data']['creators'][1]['nfts'][0]['id']]
nft_df = pd.DataFrame(nft_list, columns = ['nft_id'])

nft_history_data = json_data['data']['creators'][1]['nfts'][0]['nftHistory']
nft_history_data

nft_history_list = list()
for nft_history_number in range(len(nft_history_data)):
    nft_history_str = json_data['data']['creators'][1]['nfts'][0]['nftHistory'][nft_history_number]['id']
    nft_history_list += [nft_history_str]
    
nft_history_df = pd.DataFrame(nft_history_list, columns = ['nft_history_id'])
nft_history_df

nft_df

combined_df = pd.concat([nft_df,nft_history_df], axis = 1)
combined_df.fillna(method = 'ffill', inplace = True)

combined_df

,nft_id,nft_history_id
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x1587f580218abb040ebdc502d7eb0b78265f34e7b8ac...
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x1587f580218abb040ebdc502d7eb0b78265f34e7b8ac...
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x1febc90893e04d36e990d84d305e27b9298511b4b01b...
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x284aa8be47571cf49ec7155e4c9c1b0a052f1199e87f...
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x325741552edb163155f74afd0f6d2c1be344005da464...
5,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x36aae5236169f388b56962ba426c4163aac25592083e...
6,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x947d74553385765d1789d02f290ce01916622428274b...
7,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0xa0a30483c68eb3b0381d8d60a183cdae24c1f2ba96b9...
8,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0xb6d38adb920650910d1d7221a33735c943f649c9ee14...
9,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0xc3b4cb4484652045703c6e3713775e1026c5d0279ea9...


## method from bottom up: adding for loop for nft id

In [98]:
nft_data = json_data['data']['creators'][1]['nfts'][0:3]
nft_data

df_column_names = ['nft_id','nft_history_id']
nft_nft_history_df = pd.DataFrame(columns = df_column_names)

#nft_list = list()

for nft in range(len(nft_data)):
    nft_str = json_data['data']['creators'][1]['nfts'][nft]['id']
    nft_list = [nft_str]
    
    nft_history_data = json_data['data']['creators'][1]['nfts'][nft]['nftHistory']
    nft_history_list = list ()
    for nft_history in range(len(nft_history_data)):
        nft_history_str = nft_history_data[nft_history]['id']
        nft_history_list += [nft_history_str]
        
    nft_history_df = pd.DataFrame(nft_history_list, columns = ['nft_history_id'])
    nft_df = pd.DataFrame(nft_list, columns = ['nft_id'])
    combined_df = pd.concat([nft_df,nft_history_df],axis = 1)
    combined_df.fillna(method='ffill', inplace = True)
    
    nft_nft_history_df = pd.concat([nft_nft_history_df,combined_df])

nft_nft_history_df

,nft_id,nft_history_id
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x1587f580218abb040ebdc502d7eb0b78265f34e7b8ac...
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x1587f580218abb040ebdc502d7eb0b78265f34e7b8ac...
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x1febc90893e04d36e990d84d305e27b9298511b4b01b...
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x284aa8be47571cf49ec7155e4c9c1b0a052f1199e87f...
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x325741552edb163155f74afd0f6d2c1be344005da464...
5,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x36aae5236169f388b56962ba426c4163aac25592083e...
6,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0x947d74553385765d1789d02f290ce01916622428274b...
7,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0xa0a30483c68eb3b0381d8d60a183cdae24c1f2ba96b9...
8,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0xb6d38adb920650910d1d7221a33735c943f649c9ee14...
9,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,0xc3b4cb4484652045703c6e3713775e1026c5d0279ea9...


## method from bottom up: adding for loop for creator

In [111]:
creator_data = json_data['data']['creators']

df_column_names = ['nft_id','nft_history_id']
nft_nft_history_df = pd.DataFrame(columns = df_column_names)

for creator in range(len(creator_data)):
    
    nft_data = creator_data[creator]['nfts']
    
    for nft in range(len(nft_data)):
        nft_str = nft_data[nft]['id']
        nft_list = [nft_str]
    
        nft_history_data = nft_data[nft]['nftHistory']
        nft_history_list = list ()
        for nft_history in range(len(nft_history_data)):
            nft_history_str = nft_history_data[nft_history]['id']
            nft_history_list += [nft_history_str]
        
        nft_history_df = pd.DataFrame(nft_history_list, columns = ['nft_history_id'])
        nft_df = pd.DataFrame(nft_list, columns = ['nft_id'])
        combined_df = pd.concat([nft_df,nft_history_df],axis = 1)
        combined_df.fillna(method='ffill', inplace = True)
    
        nft_nft_history_df = pd.concat([nft_nft_history_df,combined_df])

nft_nft_history_df.reset_index
#nft_nft_history_df.head(30)
#nft_nft_history_df[nft_nft_history_df['nft_id']=="0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995"]

,nft_id,nft_history_id
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...
5,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0dfd065060f31f68adc38a0a473c94e4e0783fa9cfe5...
6,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0f44b22633d58699c089507fe63e84a20084ce39771c...
7,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0f908b44547418dea157dec80187f372d0a99527d223...
8,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x1b0e551f104b1c9cd5359e84626afe7b5ca239a66134...
9,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x23022e2487f93d40f6e9b2e80321883823d9613479aa...


In [29]:
creator_dict = json_data['data']['creators']
df_column_names = ['nft_id','nft_history_id']
nft_nft_history_df = pd.DataFrame(columns = df_column_names)

for creator_number in range(len(creator_dict)):
    
#    creator_list = list()
#    creator_str = json_data['data']['creators'][creator_number]['id']
#    creator_list += [creator_str]
#    creator_df = pd.DataFrame(creator_list, columns = ['creator_id'])
    
    
    nft_data = json_data['data']['creators'][creator_number]['nfts']
    nft_list = list()
    for nft_number in range(len(nft_data)):
        nft_str = json_data['data']['creators'][creator_number]['nfts'][nft_number]['id']
        nft_list += [nft_str]
        
        nft_history_data = json_data['data']['creators'][creator_number]['nfts'][nft_number]['nftHistory']
        nft_history_list = list()
        for nft_history_number in range(len(nft_history_data)):
            nft_history_str = json_data['data']['creators'][creator_number]['nfts'][nft_number]['nftHistory'][nft_history_number]['id']
            nft_history_list += [nft_history_str]
        
        nft_history_df = pd.DataFrame(nft_history_list, columns = ['nft_history_id'])
        nft_df = pd.DataFrame(nft_list,columns = ['nft_id'])
    
    
    
        combined_df = pd.concat([nft_df, nft_history_df], axis = 1)
        combined_df.fillna(method='ffill', inplace = True)
    
    nft_nft_history_df = pd.concat([nft_nft_history_df, combined_df])
    
nft_nft_history_df.describe().T

,count,unique,top,freq
nft_id,189,123,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,36
nft_history_id,189,95,0xbb861d10a4607c52d68dfced57474387ee34171b5304...,89


In [34]:
nft_df

,nft_id
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-16505
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-23473
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-30012
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-34776
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-35143
5,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-35456
6,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-46366
7,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-54189
8,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-61612
9,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-67662


In [31]:
nft_history_df

,nft_history_id
0,0x0e6d669a62287c6d0d22d1853eb9c120d39a9eacf6b7...
1,0x16d0d9101ddea7a446ebad450b466b6a38306164b8f1...
2,0x217cbe62465a3eae6583a49931f7087de7eedd9c6fff...
3,0x418219bda6a2df73f443c42718cae6fe8cb0958af640...
4,0x418219bda6a2df73f443c42718cae6fe8cb0958af640...
5,0x6e0b0af73fdd14ecd6e46af4a5a67f5e93006e8deeb8...
6,0xabc14b0f840995d32ca8c0675e923d720f0a0ebeded9...
7,0xb23a51868e4ce1ddb208ea191cfbe06fef9167f39575...
8,0xd878d0d7304b664a9e1a1ad723fa3c73c21c682f88dc...


In [32]:
nft_nft_history_df

,nft_id,nft_history_id
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...
...,...,...
10,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-67664,0xd878d0d7304b664a9e1a1ad723fa3c73c21c682f88dc...
11,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-76331,0xd878d0d7304b664a9e1a1ad723fa3c73c21c682f88dc...
12,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-9185,0xd878d0d7304b664a9e1a1ad723fa3c73c21c682f88dc...
13,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-9245,0xd878d0d7304b664a9e1a1ad723fa3c73c21c682f88dc...


In [23]:
json_data['data']['creators']

[{'nfts': [{'id': '0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437',
    'nftHistory': [{'id': '0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd0bb891e6d372eb4d70dd-171-Bid'},
     {'id': '0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e59004711ca4ee17d45e5d-305-Listed'},
     {'id': '0x0708e2f65b149a1b6b429dbe366bd590786e86724c810badadec435715075140-95-Bid'},
     {'id': '0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057fafb7047c905ff67d1dd-17-Bid'},
     {'id': '0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5e829d07cd8e70fbf1f11-290-Bid'},
     {'id': '0x0dfd065060f31f68adc38a0a473c94e4e0783fa9cfe5937bf917e1a3d8132f9a-379-Bid'},
     {'id': '0x0f44b22633d58699c089507fe63e84a20084ce39771cb8329e7604f41ea3de6a-60-Bid'},
     {'id': '0x0f908b44547418dea157dec80187f372d0a99527d22369689c2fc761ddb17f85-287-Bid'},
     {'id': '0x1b0e551f104b1c9cd5359e84626afe7b5ca239a66134476485e1b428bfce9561-345-Bid'},
     {'id': '0x23022e2487f93d40f6e9b2e80321883823d9613479aa54559b5103bd64882346-2-Bid'},
     {'i

## Combining the tables into one single flat table

In [ ]:
nft_df1_connected = pd.merge(nft_df1,table12_connector, how = "left", left_on = "id", right_on = "nft")
nft_df1_connected.pop("id")
nft_df1_connected.head()

In [ ]:
nfthist_df_connected = pd.merge(nfthist_df,table23_connector, how = "left", left_on = "id", right_on = "nftHistoryid")
nfthist_df_connected.pop("id")
nfthist_df_connected.head()

In [ ]:
table12_connected =  pd.merge(creator_df, nft_df1_connected, how = "left", left_on = "id", right_on = "creators_0_id", suffixes = ["_creator","_nft"])
final_table = pd.merge(table12_connected, nfthist_df_connected, how = "left", left_on = "nft", right_on = "creators_0_nfts_0_id")

In [ ]:
final_table.pop("creators_0_id")
final_table.pop("creators_0_nfts_0_id")
final_table.shape

In [ ]:
final_table.describe().T

In [ ]:
final_table.head()

## Conclusion

After creating the flat table, a total of 18 non null columns area available, of which 15 are non ID related.

3 additional columns on nft metadata ("name", "description", "image") are available but further data gathering is needed. (web-scraping and text processing?).

https://fnd.dev/docs/fnd-subgraph
"you can read the metadata JSON from https://ipfs.io/ipfs/${nft.tokenIPFSPath}"



In [ ]:
print(creator_df.shape)
print(nft_df1.shape)
print(nft_df1_connected.shape)
print(nfthist_df.shape)
print(nfthist_df_connected.shape)